In [6]:
import pandas as pd
import numpy as np
import preprocessing as pre

## 3 kadencja: 1997 - 2001

In [2]:
xml_file_path = '.\Sejm-20240316T162344Z-001\Sejm\GLO3k1p119p_i\GLO3k1p119p_i.xml'
csv_file = 'GLO3k1p119p_i.csv'

In [3]:
df_all = pre.save_xml_as_csv(xml_file_path, csv_file)

In [4]:
df_all.head()

,vote_id,data_godzina,Nrleg,NazwiskoImie,Glos,Klub
0,3001001,1997-10-20 12:51:54,1,Adamczyk Franciszek,Za,AWS
1,3001001,1997-10-20 12:51:54,2,Adamska-Wedler Elżbieta,Za,AWS
2,3001001,1997-10-20 12:51:54,3,Adamski Władysław,Za,SLD
3,3001001,1997-10-20 12:51:54,4,Ajchler Romuald,Za,SLD
4,3001001,1997-10-20 12:51:54,5,Anusz Andrzej,Za,AWS


In [7]:
assert np.all(df_all.dtypes.values == [np.dtype('int32'), np.dtype('<M8[ns]'), np.dtype('int32'), np.dtype('O'), np.dtype('O'), np.dtype('O')])
assert df_all.isnull().sum().sum() == 0
assert df_all[df_all.duplicated()].size == 0

In [8]:
df_all['data_godzina'].min(), df_all['data_godzina'].max()

(Timestamp('1997-10-20 12:51:54'), Timestamp('2001-09-18 16:26:05'))

In [9]:
y_m_list = pd.date_range(df_all['data_godzina'].min(), df_all['data_godzina'].max(), 
              freq='M').strftime("%Y-%m").tolist()

y_m_list.append(df_all['data_godzina'].max().strftime("%Y-%m"))

In [10]:
y_m_list[0], y_m_list[-1]

('1997-10', '2001-09')

In [11]:
y_m_list = [x.split('-') for x in y_m_list]

In [12]:
df_deputies = pre.assign_node_ids(df_all)

In [13]:
for year, month in y_m_list:
    df_m = df_all[(df_all['data_godzina'].dt.month == int(month)) & (df_all['data_godzina'].dt.year == int(year))]
    
    if len(df_m) == 0:
        print(year, month, 'no votings')
        continue
    
    df_m = pd.merge(df_m[['vote_id', 'data_godzina', 'Nrleg', 'NazwiskoImie', 'Glos', 'Klub']], df_deputies[['Nrleg', 'NazwiskoImie', 'node_id']], on=['Nrleg', 'NazwiskoImie'], how='left')
    df_node_atr = pre.assign_attributes(df_m)
    
    edge_matrix = pre.egde_weights(df_m)
    
    np.savetxt(f"adjacency_matrices/matrix_{year}_{month}.csv", edge_matrix, delimiter=",")

100%|████████████████████████████████████████████████████████████████████████████████| 159/159 [00:12<00:00, 12.65it/s]
0it [00:00, ?it/s]
C:\Users\Kasia\Desktop\magisterka\preprocessing.py:168: RuntimeWarning: invalid value encountered in divide
  return common_votes / len(vote_ids)
100%|████████████████████████████████████████████████████████████████████████████████| 301/301 [00:33<00:00,  9.01it/s]
0it [00:00, ?it/s]
C:\Users\Kasia\Desktop\magisterka\preprocessing.py:168: RuntimeWarning: invalid value encountered in divide
  return common_votes / len(vote_ids)
100%|████████████████████████████████████████████████████████████████████████████████| 248/248 [00:22<00:00, 11.21it/s]


In [16]:
for year, month in y_m_list:
    df_m = df_all[(df_all['data_godzina'].dt.month == int(month)) & (df_all['data_godzina'].dt.year == int(year))]
    if len(df_m) == 0:
        print(year, month)

1999 08
2000 08
